In [ ]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> d
    Error loading d: Package 'd' not found in index

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> all
Command 'all' unrecognized

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
   

True

In [ ]:
from google.colab import files
files.upload()

Saving Intents.json to Intents.json


{'Intents.json': b'{\r\n      "intents": [\r\n            {\r\n                  "intent": "Greeting",\r\n                  "text": [\r\n                        "Hi",\r\n                        "Hi there",\r\n                        "Hola",\r\n                        "Hello",\r\n                        "Hello there",\r\n                        "Hya",\r\n                        "Hya there"\r\n                  ],\r\n                  "responses": [\r\n                        "Hi human, please tell me your GeniSys user",\r\n                        "Hello human, please tell me your GeniSys user",\r\n                        "Hola human, please tell me your GeniSys user"\r\n                  ],\r\n                  "extension":  {\r\n                        "function": "",\r\n                        "entities": false,\r\n                        "responses": []\r\n                  },\r\n                  "context":  {\r\n                        "in": "",\r\n                        "out": "G

In [ ]:
import nltk
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

import numpy
import tensorflow as tf
import random
import json
import pickle

In [ ]:
with open("Intents.json") as file:
    data = json.load(file)


In [ ]:
words = []
labels = []
docs_x = []
docs_y = []
for intentt in data["intents"]:
    for pattern in intentt["text"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intentt["intent"])

    if intentt["intent"] not in labels:
        labels.append(intentt["intent"])

words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)


training = numpy.array(training)
output = numpy.array(output)


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, input_shape= (len(training[0]),)),
    tf.keras.layers.Dense(8),
    tf.keras.layers.Dense(len(output[0]), activation="softmax"),
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(training, output, epochs=100, batch_size=8)

Epoch 1/100
18/18 [==============================] - 3s 2ms/step - loss: 3.1285 - accuracy: 0.0097 
Epoch 2/100
18/18 [==============================] - 0s 2ms/step - loss: 3.0883 - accuracy: 0.0322
Epoch 3/100
18/18 [==============================] - 0s 2ms/step - loss: 3.0601 - accuracy: 0.0317
Epoch 4/100
18/18 [==============================] - 0s 2ms/step - loss: 3.0078 - accuracy: 0.0763
Epoch 5/100
18/18 [==============================] - 0s 2ms/step - loss: 2.9581 - accuracy: 0.0809
Epoch 6/100
18/18 [==============================] - 0s 2ms/step - loss: 2.9244 - accuracy: 0.1320
Epoch 7/100
18/18 [==============================] - 0s 2ms/step - loss: 2.8674 - accuracy: 0.1897
Epoch 8/100
18/18 [==============================] - 0s 2ms/step - loss: 2.8092 - accuracy: 0.2580
Epoch 9/100
18/18 [==============================] - 0s 2ms/step - loss: 2.7675 - accuracy: 0.2629
Epoch 10/100
18/18 [==============================] - 0s 2ms/step - loss: 2.7265 - accuracy: 0.2964
Epoch 11

In [ ]:
model.save("model.keras")

In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break
        bg = bag_of_words(inp, words)
        bg = numpy.reshape(bg, (1, bg.shape[0]))
        results = model.predict([bg])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['intent'] == tag:
                responses = tg['responses']

        print(random.choice(responses))


In [ ]:
chat()

Start talking with the bot (type quit to stop)!
You: Hey
Have a nice day
You: Whats your name
You may call me Geni
You: I am Akshay
Cool! Hello <HUMAN>, what can I do for you?
You: How is the day
Let me see
You: q
See you later
